<a href="https://colab.research.google.com/github/Bonjiang/Final_datasci/blob/main/HHA_550_Bonnie_Jiang_HW_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HHA 550 Diabetes Dataset

## Context

##### The dataset represents ten years (1999-2008) of clinical care at 130 US hospitals and integrated delivery networks. Each row concerns hospital records of patients diagnosed with diabetes, who underwent laboratory, medications, and stayed up to 14 days. The goal is to determine the early readmission of the patient within 30 days of discharge. The problem is important for the following reasons. Despite high-quality evidence showing improved clinical outcomes for diabetic patients who receive various preventive and therapeutic interventions, many patients do not receive them. This can be partially attributed to arbitrary diabetes management in hospital environments, which fail to attend to glycemic control. Failure to provide proper diabetes care not only increases the managing costs for the hospitals (as the patients are readmitted) but also impacts the morbidity and mortality of the patients, who may face complications associated with diabetes.

## Import libraries

In [ ]:
!pip install seaborn
!pip install missingno
!pip install xgboost
!pip install catboost
!pip install regex
!pip install sklearn
!pip install pandas
!pip install numpy
!pip install imblearn
!pip install lightgbm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 8.0 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
import pandas as pd   # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np   # linear algebra
import matplotlib.pyplot as plt  #graphs and plots
import seaborn as sns   #data visualizations
import plotly.express as px  # Import Plotly Express
import csv # Some extra functionalities for csv  files - reading it as a dictionary
from lightgbm import LGBMClassifier #sklearn is for machine learning and statistical modeling including classification, regression, clustering and dimensionality reduction
from sklearn.model_selection import train_test_split, cross_validate   #break up dataset into train and test sets

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler


# importing python library for working with missing data
import missingno as msno
# To install missingno use: !pip install missingno
import re    # This library is used to perform regex pattern matching

# import various functions from sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import GradientBoostingClassifier
from catboost import CatBoostClassifier
import xgboost as xgb
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, classification_report, make_scorer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split

In [ ]:
import plotly
import plotly.express as px
import plotly.graph_objs as go
import plotly.offline as py
from plotly.offline import iplot
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import warnings
warnings.filterwarnings("ignore")

## Exploratory Data Analysis (EDA)

### Loading CSV

In [ ]:
df = pd.read_csv('diabetic_data.csv')
df

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101761,443847548,100162476,AfricanAmerican,Male,[70-80),?,1,3,7,3,...,No,Down,No,No,No,No,No,Ch,Yes,>30
101762,443847782,74694222,AfricanAmerican,Female,[80-90),?,1,4,5,5,...,No,Steady,No,No,No,No,No,No,Yes,NO
101763,443854148,41088789,Caucasian,Male,[70-80),?,1,1,7,1,...,No,Down,No,No,No,No,No,Ch,Yes,NO
101764,443857166,31693671,Caucasian,Female,[80-90),?,2,3,7,10,...,No,Up,No,No,No,No,No,Ch,Yes,NO


### Looking at Metrics

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101766 entries, 0 to 101765
Data columns (total 50 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   encounter_id              101766 non-null  int64 
 1   patient_nbr               101766 non-null  int64 
 2   race                      101766 non-null  object
 3   gender                    101766 non-null  object
 4   age                       101766 non-null  object
 5   weight                    101766 non-null  object
 6   admission_type_id         101766 non-null  int64 
 7   discharge_disposition_id  101766 non-null  int64 
 8   admission_source_id       101766 non-null  int64 
 9   time_in_hospital          101766 non-null  int64 
 10  payer_code                101766 non-null  object
 11  medical_specialty         101766 non-null  object
 12  num_lab_procedures        101766 non-null  int64 
 13  num_procedures            101766 non-null  int64 
 14  num_

In [ ]:
dtype_counts = df.dtypes.value_counts()
print(dtype_counts)

object    37
int64     13
dtype: int64


In [ ]:
# Checking number of rows and columns
print("Shape of original DataFrame:", df.shape)

Shape of original DataFrame: (101766, 50)


### Identify and Handling Missing/Unknown Values in a Dataset

In [ ]:
df.replace('?', np.nan, inplace=True)

In [ ]:
unknown_values = df.isnull().sum()
unknown_values

encounter_id                    0
patient_nbr                     0
race                         2273
gender                          0
age                             0
weight                      98569
admission_type_id               0
discharge_disposition_id        0
admission_source_id             0
time_in_hospital                0
payer_code                  40256
medical_specialty           49949
num_lab_procedures              0
num_procedures                  0
num_medications                 0
number_outpatient               0
number_emergency                0
number_inpatient                0
diag_1                         21
diag_2                        358
diag_3                       1423
number_diagnoses                0
max_glu_serum                   0
A1Cresult                       0
metformin                       0
repaglinide                     0
nateglinide                     0
chlorpropamide                  0
glimepiride                     0
acetohexamide 

In [ ]:
def missing (df):
    missing_number = df.isnull().sum().sort_values(ascending=False)
    missing_percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
    missing_values = pd.concat([missing_number, missing_percent], axis=1, keys=['Missing_Number', 'Missing_Percent'])
    return missing_values

missing(df)

,Missing_Number,Missing_Percent
weight,98569,0.968585
medical_specialty,49949,0.490822
payer_code,40256,0.395574
race,2273,0.022336
diag_3,1423,0.013983
diag_2,358,0.003518
diag_1,21,0.000206
encounter_id,0,0.000000
tolazamide,0,0.000000
glyburide,0,0.000000


In [ ]:
# Dropping NaN values from columns with NaN values, I didn't want to drop the entire column
columns_to_drop_na = ['race',
                      'payer_code',
                      'medical_specialty',
                      'diag_1',
                      'diag_2',
                      'diag_3']
df= df.dropna(subset=columns_to_drop_na)

# Check the  shape of the cleaned DataFrame, data points and rows to double check
print("Shape of cleaned DataFrame:", df.shape)


Shape of cleaned DataFrame: (26755, 50)


In [ ]:
# Dropping columns related to "id", since I don't think those are necessary
# Also dropped weight because 96.8% of its values were missing
columns_to_drop = ['encounter_id',
                   'patient_nbr',
                   'admission_type_id',
                   'discharge_disposition_id',
                   'admission_source_id',
                   'weight']
df = df.drop(columns=columns_to_drop)

# Check the shape of the cleaned DataFrame, data points and rows to double check
print("DataFrame after dropping columns:")
print(df)


DataFrame after dropping columns:
             race  gender       age  time_in_hospital payer_code  \
20446   Caucasian  Female   [70-80)                 7         MC   
20737   Caucasian  Female   [60-70)                 3         MC   
20824   Caucasian  Female  [90-100)                 4         MC   
21083   Caucasian    Male   [70-80)                10         MC   
23879   Caucasian  Female   [70-80)                12         UN   
...           ...     ...       ...               ...        ...   
101735  Caucasian    Male   [70-80)                 6         MC   
101743  Caucasian    Male   [80-90)                 3         MC   
101747  Caucasian  Female   [70-80)                 8         MC   
101749  Caucasian  Female   [70-80)                 3         MC   
101764  Caucasian  Female   [80-90)                10         MC   

                 medical_specialty  num_lab_procedures  num_procedures  \
20446   Orthopedics-Reconstructive                  58               2   
2

In [ ]:
# Check for NaN values in each column
nan_counts = df.isnull().sum()

# Display columns with NaN values and their counts
print("Columns with NaN values and their counts:")
print(nan_counts)

# Verify if NaN values are present in the entire DataFrame
total_nan_count = nan_counts.sum()
if total_nan_count == 0:
    print("No NaN values are present in the DataFrame.")
else:
    print(f"Total NaN values in the DataFrame: {total_nan_count}")


Columns with NaN values and their counts:
race                        0
gender                      0
age                         0
time_in_hospital            0
payer_code                  0
medical_specialty           0
num_lab_procedures          0
num_procedures              0
num_medications             0
number_outpatient           0
number_emergency            0
number_inpatient            0
diag_1                      0
diag_2                      0
diag_3                      0
number_diagnoses            0
max_glu_serum               0
A1Cresult                   0
metformin                   0
repaglinide                 0
nateglinide                 0
chlorpropamide              0
glimepiride                 0
acetohexamide               0
glipizide                   0
glyburide                   0
tolbutamide                 0
pioglitazone                0
rosiglitazone               0
acarbose                    0
miglitol                    0
troglitazone                

## Visualization of Readmittance of patients with diabetes

In [ ]:
# Create a histogram plot using Plotly Express
fig = px.histogram(df, x="readmitted", title='Readmitted of Patients with Diabetes', width=500, height=500)

# Show the plot
fig.show()

## Numerical Features

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26755 entries, 20446 to 101764
Data columns (total 44 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   race                      26755 non-null  object
 1   gender                    26755 non-null  object
 2   age                       26755 non-null  object
 3   time_in_hospital          26755 non-null  int64 
 4   payer_code                26755 non-null  object
 5   medical_specialty         26755 non-null  object
 6   num_lab_procedures        26755 non-null  int64 
 7   num_procedures            26755 non-null  int64 
 8   num_medications           26755 non-null  int64 
 9   number_outpatient         26755 non-null  int64 
 10  number_emergency          26755 non-null  int64 
 11  number_inpatient          26755 non-null  int64 
 12  diag_1                    26755 non-null  object
 13  diag_2                    26755 non-null  object
 14  diag_3           

In [ ]:
# Numerical variables (int64):
numerical = ['time_in_hospital',
             'num_lab_procedures',
             'num_procedures',
             'num_medications',
             'number_outpatient',
             'number_emergency',
             'number_inpatient',
             'number_diagnoses']

In [ ]:
df[numerical].describe()

,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses
count,26755.000000,26755.000000,26755.000000,26755.000000,26755.000000,26755.000000,26755.000000,26755.000000
mean,4.317922,40.756681,1.475575,16.246720,0.290712,0.311755,0.670641,7.355522
std,2.949791,19.965546,1.749729,8.609362,0.998707,1.437214,1.367311,1.895473
min,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,3.000000
25%,2.000000,29.000000,0.000000,10.000000,0.000000,0.000000,0.000000,6.000000
50%,4.000000,42.000000,1.000000,15.000000,0.000000,0.000000,0.000000,8.000000
75%,6.000000,54.000000,2.000000,20.000000,0.000000,0.000000,1.000000,9.000000
max,14.000000,132.000000,6.000000,81.000000,38.000000,76.000000,16.000000,16.000000


In [ ]:
# readmitted
print (f'{round(df["readmitted"].value_counts(normalize=True)*100,2)}')
fig = px.histogram(df, x="readmitted", title='readmitted', width=500, height=500)
fig.show()

NO     56.64
>30    32.68
<30    10.68
Name: readmitted, dtype: float64


In [ ]:
# Convert categorical data to numerical data
df = pd.get_dummies(df, drop_first = False)

In [ ]:
df.columns

Index(['time_in_hospital', 'num_lab_procedures', 'num_procedures',
       'num_medications', 'number_outpatient', 'number_emergency',
       'number_inpatient', 'number_diagnoses', 'race_AfricanAmerican',
       'race_Asian',
       ...
       'metformin-rosiglitazone_No', 'metformin-pioglitazone_No',
       'metformin-pioglitazone_Steady', 'change_Ch', 'change_No',
       'diabetesMed_No', 'diabetesMed_Yes', 'readmitted_<30', 'readmitted_>30',
       'readmitted_NO'],
      dtype='object', length=1903)

In [ ]:
# Breaking the data up into Train & Test
train_df, valid_df, test_df = np.split(df.sample(frac=1, random_state=42),
 [int(.7*len(df)), int(0.85*len(df))])
train_df = train_df.reset_index(drop = True)
valid_df = valid_df.reset_index(drop = True)
test_df = test_df.reset_index(drop = True)

In [ ]:
df.columns

Index(['time_in_hospital', 'num_lab_procedures', 'num_procedures',
       'num_medications', 'number_outpatient', 'number_emergency',
       'number_inpatient', 'number_diagnoses', 'race_AfricanAmerican',
       'race_Asian',
       ...
       'metformin-rosiglitazone_No', 'metformin-pioglitazone_No',
       'metformin-pioglitazone_Steady', 'change_Ch', 'change_No',
       'diabetesMed_No', 'diabetesMed_Yes', 'readmitted_<30', 'readmitted_>30',
       'readmitted_NO'],
      dtype='object', length=1903)

In [ ]:
df['readmitted_<30'].value_counts()

0    23898
1     2857
Name: readmitted_<30, dtype: int64

In [ ]:
def calc_prevalence(y_actual):

 '''
 This function is to understand the ratio/distribution of the classes that we are going to predict for.

 Params:
 1. y_actual: The target feature

 Return:
 1. (sum(y_actual)/len(y_actual)): The ratio of the postive class in the complete data.
 '''

 return (sum(y_actual)/len(y_actual))

In [ ]:
# split the training data into positive and negative
rows_pos = train_df['readmitted_<30'] == 1
df_train_pos = train_df.loc[rows_pos]
df_train_neg = train_df.loc[~rows_pos]

# merge the balanced data
diabetes_df_balanced = pd.concat([df_train_pos, df_train_neg.sample(n=len(df_train_pos), random_state=111)], axis=0)

# shuffle the order of training samples
diabetes_df_balanced = diabetes_df_balanced.sample(n=len(diabetes_df_balanced), random_state=42).reset_index(drop=True)

print('Train balanced prevalence(n = %d): %.3f' % (len(diabetes_df_balanced), calc_prevalence(diabetes_df_balanced['readmitted_<30'].values)))

Train balanced prevalence(n = 4032): 0.500


In [ ]:
X_train = diabetes_df_balanced.drop('readmitted_<30',axis=1)

y_train = diabetes_df_balanced['readmitted_<30']

X_valid = valid_df.drop('readmitted_<30',axis=1)

y_valid = valid_df['readmitted_<30']

X_test = test_df.drop('readmitted_<30',axis=1)

y_test = test_df['readmitted_<30']

In [ ]:
scaler=StandardScaler()
X_train[['time_in_hospital', 'num_lab_procedures', 'num_procedures', 'num_medications', 'number_outpatient', 'number_emergency', 'number_inpatient', 'number_diagnoses' ]] = pd.DataFrame(scaler.fit_transform(X_train[['time_in_hospital', 'num_lab_procedures', 'num_procedures', 'num_medications', 'number_outpatient', 'number_emergency', 'number_inpatient', 'number_diagnoses']]),columns=['time_in_hospital', 'num_lab_procedures', 'num_procedures', 'num_medications', 'number_outpatient', 'number_emergency', 'number_inpatient', 'number_diagnoses'])
X_valid[['time_in_hospital', 'num_lab_procedures', 'num_procedures', 'num_medications', 'number_outpatient', 'number_emergency', 'number_inpatient', 'number_diagnoses']] = pd.DataFrame(scaler.transform(X_valid[['time_in_hospital', 'num_lab_procedures', 'num_procedures', 'num_medications', 'number_outpatient', 'number_emergency', 'number_inpatient', 'number_diagnoses']]),columns=['time_in_hospital', 'num_lab_procedures', 'num_procedures', 'num_medications', 'number_outpatient', 'number_emergency', 'number_inpatient', 'number_diagnoses'])
X_test[['time_in_hospital', 'num_lab_procedures', 'num_procedures', 'num_medications', 'number_outpatient', 'number_emergency', 'number_inpatient', 'number_diagnoses']] = pd.DataFrame(scaler.transform(X_test[['time_in_hospital', 'num_lab_procedures', 'num_procedures', 'num_medications', 'number_outpatient', 'number_emergency', 'number_inpatient', 'number_diagnoses']]),columns=['time_in_hospital', 'num_lab_procedures', 'num_procedures', 'num_medications', 'number_outpatient', 'number_emergency', 'number_inpatient', 'number_diagnoses'])


In [ ]:
def calc_specificity(y_actual, y_pred, thresh):
    # calculates specificity
    return sum((y_pred < thresh) & (y_actual == 0)) /sum(y_actual ==0)

def print_report(y_actual, y_pred, thresh = 0.5):

    '''
    This function calculates all the metrics to asses the machine learning models.

    Params:
    1. y_actual: The actual values for the target variable.
    2. y_pred: The predicted values for the target variable.
    3. thresh: The threshold for the probability to be considered as a positive class. Default value 0.5

    Return:
    1. AUC
    2. Accuracy
    3. Recall
    4. Precision
    5. Specificity
    '''

    auc = roc_auc_score(y_actual, y_pred)
    accuracy = accuracy_score(y_actual, (y_pred > thresh))
    recall = recall_score(y_actual, (y_pred > thresh))
    precision = precision_score(y_actual, (y_pred > thresh))
    specificity = calc_specificity(y_actual, y_pred, thresh)
    print('AUC:%.3f'%auc)
    print('accuracy:%.3f'%accuracy)
    print('recall:%.3f'%recall)
    print('precision:%.3f'%precision)
    print('specificity:%.3f'%specificity)
    print('prevalence:%.3f'%calc_prevalence(y_actual))
    print(' ')
    return auc, accuracy, recall, precision, specificity

In [ ]:
lnr = LinearRegression()
lnr.fit(X_train, y_train)


y_valid_preds = lnr.predict(X_valid)

In [ ]:
y_valid_preds

array([ 3.33066907e-16,  5.55111512e-16,  1.00000000e+00, ...,
       -1.77635684e-15,  1.11022302e-15, -9.99200722e-16])

## Linear Regression

In [ ]:
lnr = LinearRegression()
lnr.fit(X_train, y_train)


y_valid_preds = lnr.predict(X_valid)

In [ ]:
y_valid_preds

array([[ 5.46875000e-02, -5.17578125e-02,  1.00000000e+00],
       [ 7.37304688e-01,  2.62695312e-01,  4.21884749e-15],
       [ 6.72851562e-01,  3.30078125e-01,  9.99200722e-16],
       ...,
       [ 6.68945312e-01,  3.33984375e-01,  2.33146835e-15],
       [ 6.66992188e-01,  3.32031250e-01, -7.77156117e-16],
       [ 1.48437500e-01, -1.46484375e-01,  1.00000000e+00]])

### KNN Model

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 100)
knn.fit(X_train, y_train)

knn_preds = knn.predict_proba(X_valid)[:,1]

lr_valid_auc, lr_valid_accuracy, lr_valid_recall, \
    lr_valid_precision, lr_valid_specificity = print_report(y_valid,knn_preds, 0.5)

AUC:0.898
accuracy:0.735
recall:0.883
precision:0.272
specificity:0.685
prevalence:0.107
 


## Stochastic Gradient Descent Model

In [ ]:
sgdc=SGDClassifier(loss = 'log',alpha = 0.1,random_state = 42)
sgdc.fit(X_train, y_train)

sgd_preds = sgdc.predict_proba(X_valid)[:,1]

print('Stochastic Gradient Descent')
print('Validation:')
sgdc_valid_auc, sgdc_valid_accuracy, sgdc_valid_recall, \
                sgdc_valid_precision, sgdc_valid_specificity = print_report(y_valid,sgd_preds, 0.5)

Stochastic Gradient Descent
Validation:
AUC:0.987
accuracy:0.937
recall:0.993
precision:0.631
specificity:0.931
prevalence:0.107
 


## Decision Tree

In [ ]:
dc_clf = DecisionTreeClassifier(random_state=42, max_depth = 10)
dc_clf.fit(X_train, y_train)

dc_preds_proba = dc_clf.predict_proba(X_valid)[:,1]
dc_preds = dc_clf.predict(X_valid)

lr_valid_auc, lr_valid_accuracy, lr_valid_recall, \
    lr_valid_precision, lr_valid_specificity = print_report(y_valid,dc_preds_proba, 0.5)

AUC:1.000
accuracy:1.000
recall:1.000
precision:1.000
specificity:1.000
prevalence:0.107
 


## Random Forest

In [ ]:
rf_clf = RandomForestClassifier(random_state=111, max_depth = 6)

rf_clf.fit(X_train, y_train)

rf_preds = rf_clf.predict(X_valid)
rf_preds_proba = rf_clf.predict_proba(X_valid)[:, 1]

lr_valid_auc, lr_valid_accuracy, lr_valid_recall, \
    lr_valid_precision, lr_valid_specificity = print_report(y_valid,rf_preds_proba, 0.5)

AUC:0.951
accuracy:0.903
recall:0.860
precision:0.528
specificity:0.908
prevalence:0.107
 


## Linear SVC

In [ ]:
lsvc_clf = LinearSVC(random_state=111)
lsvc_clf.fit(X_train, y_train)

lsvc_preds = lsvc_clf.decision_function(X_valid)

lr_valid_auc, lr_valid_accuracy, lr_valid_recall, \
    lr_valid_precision, lr_valid_specificity = print_report(y_valid,lsvc_preds, 0.5)

AUC:1.000
accuracy:1.000
recall:1.000
precision:1.000
specificity:1.000
prevalence:0.107
 


## Gradient Boosting Model

In [ ]:
gb_clf = GradientBoostingClassifier(n_estimators = 100, criterion='friedman_mse', learning_rate = 1.0, max_depth = 3,\
                                    random_state = 111)

gb_clf.fit(X_train, y_train)
gb_preds = gb_clf.predict(X_valid)
gb_preds_proba = gb_clf.predict_proba(X_valid)[:, 1]

lr_valid_auc, lr_valid_accuracy, lr_valid_recall, \
    lr_valid_precision, lr_valid_specificity = print_report(y_valid,gb_preds_proba, 0.5)

AUC:1.000
accuracy:1.000
recall:1.000
precision:1.000
specificity:1.000
prevalence:0.107
 


## XGB Model

In [ ]:
X_train_np2 = X_train.to_numpy()
X_valid_np2 = X_valid.to_numpy()


xgb_clf = xgb.XGBClassifier(max_depth=3, learning_rate = 1.0, use_label_encoder = False,\
                            eval_metric = 'logloss')
xgb_clf.fit(X_train_np2, y_train)

xgb_preds = xgb_clf.predict(X_valid_np2)
xgb_preds_proba = xgb_clf.predict_proba(X_valid_np2)[:, 1]

lr_valid_auc, lr_valid_accuracy, lr_valid_recall, \
    lr_valid_precision, lr_valid_specificity = print_report(y_valid,xgb_preds_proba, 0.5)

AUC:1.000
accuracy:1.000
recall:1.000
precision:1.000
specificity:1.000
prevalence:0.107
 


## Catboost Model

In [ ]:
catb=CatBoostClassifier(iterations=200, depth=3, learning_rate=1.0, random_state = 111)
catb.fit(X_train, y_train)
catb_preds = catb.predict_proba(X_valid)[:, 1]

lr_valid_auc, lr_valid_accuracy, lr_valid_recall, \
    lr_valid_precision, lr_valid_specificity = print_report(y_valid,catb_preds, 0.5)

## Hyper Parameter Tuning

In [ ]:
recall_scoring = make_scorer(recall_score)

## Decision Tree - Hyper Parameter Tuning

In [ ]:
dc_grid = {'max_features':['auto','sqrt'], # maximum number of features to use at each split
           'max_depth':range(1,11,1), # maximum depth of the tree
           'min_samples_split':range(2,10,2), # minimum number of samples to split a node
           'criterion':['gini','entropy']} # criterion for evaluating a split

dc_random = RandomizedSearchCV(estimator = dc_clf, param_distributions = dc_grid,
                               n_iter = 20, cv = 2, scoring=recall_scoring,
                               verbose = 1, random_state = 111)

dc_random.fit(X_train, y_train)

dc_random.best_params_

dc_hp_preds = dc_random.best_estimator_.predict(X_valid)
dc_hp_preds_proba = dc_random.best_estimator_.predict_proba(X_valid)[:,1]
roc_auc_score(y_valid, dc_hp_preds_proba)

Fitting 2 folds for each of 20 candidates, totalling 40 fits


0.614557968073593

In [ ]:
recall_score(y_valid, dc_hp_preds)

0.5011655011655012

## Random Forest - Hyper Parameter Tuning

In [ ]:
rf_grid = {'n_estimators':range(200,1000,200), # number of trees
           'max_features':['auto','sqrt'], # maximum number of features to use at each split
           'max_depth':range(1,11,1), # maximum depth of the tree
           'min_samples_split':range(2,10,2), # minimum number of samples to split a node
           'criterion':['gini','entropy']} # criterion for evaluating a split

rf_random = RandomizedSearchCV(estimator = rf_clf, param_distributions = rf_grid,
                               n_iter = 20, cv = 2, scoring=recall_scoring,
                               verbose = 1, random_state = 111)

rf_random.fit(X_train, y_train)

rf_random.best_params_

rf_hp_preds = rf_random.best_estimator_.predict(X_valid)
rf_hp_preds_proba = rf_random.best_estimator_.predict_proba(X_valid)[:,1]
roc_auc_score(y_valid, rf_hp_preds_proba)

Fitting 2 folds for each of 20 candidates, totalling 40 fits


0.9876841908091908

In [ ]:
recall_score(y_valid, rf_hp_preds)

0.9230769230769231

## XGBoost - Hyper Parameter Tuning

In [ ]:
xgb_grid = params = {
        'min_child_weight': [1, 5, 8, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 0.9, 1.0],
        'max_depth': [3, 4, 5]
        } # criterion for evaluating a split

xgb_random = GridSearchCV(estimator = xgb_clf, param_grid = xgb_grid,
                               cv = 2, scoring = recall_scoring,
                               verbose = 1)

xgb_random.fit(X_train, y_train)

xgb_random.best_params_

xgb_hp_preds = xgb_random.best_estimator_.predict(X_valid)
xgb_hp_preds_proba = xgb_random.best_estimator_.predict_proba(X_valid)[:,1]
roc_auc_score(y_valid, xgb_hp_preds_proba)

Fitting 2 folds for each of 720 candidates, totalling 1440 fits


ValueError: 
All the 1440 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1440 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/xgboost/core.py", line 730, in inner_f
    return func(**kwargs)
  File "/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py", line 1500, in fit
    train_dmatrix, evals = _wrap_evaluation_matrices(
  File "/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py", line 521, in _wrap_evaluation_matrices
    train_dmatrix = create_dmatrix(
  File "/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py", line 958,

In [ ]:
recall_score(y_valid, xgb_hp_preds)